# EVNTO Chatbot
We are going to create a chatbot with LangChain and Google Generative AI (I chose this over HuggingFace Hub due to the ease of deployment). To build the chatbot, we first need to change the system message and make it all about what EVNTO app can do. Then, we will create a chain (with the use of RAG) to pass the events data to the model so that it can answer questions related to the type of events that we have.

## Installation
The LangChain Google AI integration lives in the `langchain-google-genai` package:

In [ ]:
%%capture
!pip install langchain-google-genai
!pip install langchain
!pip install langchain-community
!pip install jq
!pip install faiss-cpu

Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Once you've done this set the GOOGLE_API_KEY environment variable:

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your AI API key: ")

Enter your Google AI API key: ··········


## Instantiation
Now we can instantiate our model object and generate chat completions:

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
result = model.invoke("What is the outcome of 99 divided by 22?")
print(result.content)

99 divided by 22 is **4.5**. 



In [ ]:
%%writefile events_data.json
[
  {
    "event_name": "AI Workshop",
    "date": "2024-09-30",
    "location": "Mansoura University",
    "description": "A workshop on AI and its applications."
  },
  {
    "event_name": "Data Science Seminar",
    "date": "2024-10-10",
    "location": "Cairo",
    "description": "An event about the latest trends in data science."
  }
]

Writing events_data.json


## Create a Knowledge Base

Using Langchain's document loaders and vector stores, we can create a knowledge base that the chatbot can query for relevant event data.
* **Document Loader**: Load our events data into Langchain as documents.

In [ ]:
from langchain.document_loaders import JSONLoader

# Extract the data fields as dictionaries
jq_schema = '.[] | {event_name, date, location, description}'

loader = JSONLoader(file_path="events_data.json", jq_schema=jq_schema, text_content=False)
documents = loader.load()

* **Vector Store**: to store and search through the event information. Langchain supports vector stores like FAISS or Pinecone to index the event descriptions. However, we will use FAISS.

In [ ]:
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(documents, embeddings)

## Integrate the Vector Store with our Chatbot
After loading the events into a vector store, now it's time integrate it into our chatbot pipeline. The chatbot will query the vector store to retrieve relevant event information based on the user's input.

* **Retrieval-based Chatbot**: where the chatbot can fetch data from our vector store.

In [ ]:
from langchain.chains import RetrievalQA

retriever = vector_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=model, retriever=retriever)

In [ ]:
query = "Tell me about upcoming events at Mansoura University."
result = qa_chain.invoke(query)
print(result['result'])

There is one upcoming event at Mansoura University:

* **Event Name:** AI Workshop
* **Date:** 2024-09-30
* **Location:** Mansoura University
* **Description:** A workshop on AI and its applications. 

